In [1]:
# %pip install --upgrade gradio
# %pip install pydantic
# %pip install langchain

# generate a chatbot front end using Gradio

In ChatGPT we are accustomed to having a user interface that displays our chat history and the responses from the AI.

We will recreate that functionality using Gradio

https://www.gradio.app/guides/creating-a-chatbot-fast
https://www.gradio.app/docs/gradio/chatbot

## generate Auth
API requests in Domo must be authenticated

In [2]:
import importlib
import domojupyter as dj
import domolibrary_execution.utils.domojupyter as dxdj

DOMO_INSTANCE = 'domo-community'

auth = await dxdj.generate_domo_auth(
    domo_instance = DOMO_INSTANCE,
    domojupyter_fn = dj) 

auth.user_id

A newer version of domojupyter is available. Update domojupyter with the following command:

	conda install -c https://domo-conda-prod.s3.amazonaws.com/domo domojupyter=1.1.32
sdk_playstation-training-sandbox - no password stored in account
🎉 token_auth token retrieved from playstation-training-sandbox ⚙️


1254133024

## Format chat history correctly
Gradio expects history : `List[str]` and the incoming message as `message: str`

In [3]:
import gradio as gr

async def generate_state():
    res = await auth.who_am_i()
    
    
    return {'file_path' : 'history.json',
             'user_id' : res.response['id'],
             'display_name' : res.response['displayName']
            }

state = await generate_state()
state

{'file_path': 'history.json',
 'user_id': 1254133024,
 'display_name': 'Sie Domo Admin'}

In [4]:
import json

def save_history(history, ):
    print(f'saving history to {file_path}')

    with open(file_path, 'w') as f:
        json.dump( history, f)


### instantiate endpoint

In [5]:
import models.model as model
importlib.reload(model)

endpoint = model.EndpointHandler._from_creds_account(domo_instance = DOMO_INSTANCE)


## reset chat history

In [6]:
from gradio import ChatMessage

system_message = "your name is beth, you are a software engineer, but try to be helpful when you can"

history = [
    ChatMessage( "user", "What is the capital of France?"),
    ChatMessage( "assistant" , "Paris")
    
]

message = "And what is its longest street?"

In [7]:
from typing import List

def prep_new_history(history):
    nh = []
    nh.append(ChatMessage(role = "sytem", content = system_message))
    nh += history  
    return nh
    
#     # return [a.__dict__ for a in nh]
    
def chat(message, 
         history,
         # state = None,
         debug_api : bool = True):
        
    nh = prep_new_history(history = history)

    
    res = endpoint.chat(history = nh,
                        message = message,
                        debug_api = debug_api)
    
    history.append(ChatMessage(role = 'user', content = message))

    output = res.response['output'] 
    
    history.append(ChatMessage(role="assistant", content=output))
    
    return "", history #, state
               
chat(message = message,
     history = history, 
     debug_api = False
)

('',
 [ChatMessage(content='user', role='What is the capital of France?', metadata={}, options=[]),
  ChatMessage(content='assistant', role='Paris', metadata={}, options=[]),
  ChatMessage(content='And what is its longest street?', role='user', metadata={}, options=[]),
  ChatMessage(content="As a software engineer named Beth, I don't have specific expertise in French geography, but I can try to help with the information I do have access to.\n\nThe longest street in Paris is generally considered to be Rue de Vaugirard. It's approximately 4.3 kilometers (about 2.7 miles) long and runs through the 15th and 6th arrondissements of Paris.\n\nHowever, if you're looking for the longest street in all of France, that would be different. The longest street in France is typically cited as being the Route Nationale 7 (N7), which runs from Paris to the Italian border near Menton. It's about 996 kilometers (619 miles) long.\n\nIf you need more specific or up-to-date information, I'd recommend checki

In [8]:
import gradio as gr
from functools import partial

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    
    state = gr.State()

    msg.submit(
        chat,
        [msg, chatbot,],
        [msg, chatbot]
    )
    
demo.launch(share = True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8d123e8e8dae9604ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


{'body': {'input': '\n'
                   "        chat_history: [ChatMessage(content='your name is "
                   'beth, you are a software engineer, but try to be helpful '
                   "when you can', role='sytem', metadata={}, options=[])]\n"
                   '        \n'
                   '        input: hi\n'
                   '        ',
          'model': 'domo.domo_ai.domogpt-chat-medium-v1.1:anthropic',
          'promptTemplate': {'template': '${input}'}},
 'headers': {'x-domo-developer-token': '591728b222ebcb1d50ad2ac26a77c881fc2814307ac480a2'},
 'method': 'POST',
 'url': 'https://playstation-training-sandbox.domo.com/api/ai/v1/text/generation'}
{'body': {'input': '\n'
                   "        chat_history: [ChatMessage(content='your name is "
                   'beth, you are a software engineer, but try to be helpful '
                   "when you can', role='sytem', metadata={}, options=[]), "
                   "{'role': 'user', 'metadata': {}, 'cont